In [ ]:
from sklearn_crfsuite import CRF
import re
from sklearn.model_selection import train_test_split
from sklearn.metrics._classification import classification_report
from sklearn.naive_bayes import MultinomialNB
import sklearn_crfsuite
from sklearn_crfsuite import scorers
from sklearn_crfsuite import metrics
import pickle

In [ ]:
sentenceNtag = []

In [ ]:
with open("D:/PythonProject/CRF_tokenize/IOB_data/train1.txt", encoding='utf8') as file:
    for line in file:
        cnt = 0
        word_buff = ""
        tag_buff = ""
        
        while line[cnt].isspace() != True:
            word_buff += line[cnt]
            cnt += 1
        
        cnt = len(line) - 2 
        while line[cnt].isspace() != True: 
            tag_buff += line[cnt]
            cnt -= 1
        tag_buff = tag_buff[::-1]
        
        sentenceNtag.append((word_buff, tag_buff))


In [ ]:
sentences = []

temp = []
for i in range(len(sentenceNtag)):
    if sentenceNtag[i][0] != '.':
        temp.append(sentenceNtag[i])
    else:
        sentences.append(temp)
        temp = []

In [ ]:
print(len(sentences))

In [ ]:
import glob
import os

path = 'D:\PythonProject\CRF_tokenize\dictionary'
filenames = glob.glob(path + "/*.txt")
dict = []

for file in filenames:
    with open(file= file, mode= 'r', encoding='utf-8') as file_in:
        for line in file_in:
            
            line = line.replace("\n" or " \n" , "")
            dict.append(line)


In [31]:
len(dict)

91969

In [32]:
# check in dictionary
def check_dict(word, dict):
    for w in dict:
        if w == word:
            return 1
    return 0


In [35]:
def word2features(sent, i):
    word = sent[i][0]
    
    features = {
        'bias': 1.0,
        'word.lower()': word.lower(),
        'word[-3:]': word[-3:],
        'word[-2:]': word[-2:],
        'word.isupper()': word.isupper(),
        'word.istitle()': word.istitle(),
        'word.isdigit()': word.isdigit(),
    }
    if i > 1:
        word1 = sent[i-1][0]
        features.update({
            '-1:word.lower()': word1.lower(),
            '-1:word.istitle()': word1.istitle(),
            '-1:word.isupper()': word1.isupper(),
            '-1:word.check_dict()': check_dict(" ".join([word1, word]), dict),
            '-2:word.lower()': sent[i-2][0].lower(),
            '-2:word.istitle()': sent[i-2][0].istitle(),
            '-2:word.isupper()': sent[i-2][0].isupper(),

        })
    else:
        features['BOS'] = True

    if i < len(sent)-2:
        word1 = sent[i+1][0]
        features.update({
            '+1:word.lower()': word1.lower(),
            '+1:word.istitle()': word1.istitle(),
            '+1:word.isupper()': word1.isupper(),
            '+1:word.check_dict()': check_dict(" ".join([word, word1]), dict),
            '+2:word.lower()': sent[i+2][0].lower(),
            '+2:word.istitle()': sent[i+2][0].istitle(),
            '+2:word.isupper()': sent[i+2][0].isupper(),
        })
    else:
        features['EOS'] = True

    if i>1 and i < (len(sent)-2):
        word_prev = sent[i-1][0]
        word_next = sent[i+1][0]
        features.update({
            'combined_word.check_dict()': check_dict(" ".join([word_prev, word, word_next]), dict),
        })

    return features


def sent2features(sent):
    return [word2features(sent, i) for i in range(len(sent))]

def sent2labels(sent):
    return [label for token, label in sent]

In [36]:
%%time
X_train = [sent2features(s) for s in sentences]
y_train = [sent2labels(s) for s in sentences]

X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size=0.2, random_state=None)


CPU times: total: 17min 23s
Wall time: 17min 27s


In [37]:
X_train[0]

[{'bias': 1.0,
  'word.lower()': '',
  'word[-3:]': '',
  'word[-2:]': '',
  'word.isupper()': False,
  'word.istitle()': False,
  'word.isdigit()': False,
  'BOS': True,
  '+1:word.lower()': '09-02-2006',
  '+1:word.istitle()': False,
  '+1:word.isupper()': False,
  '+1:word.check_dict()': 0,
  '+2:word.lower()': '16:08:49',
  '+2:word.istitle()': False,
  '+2:word.isupper()': False},
 {'bias': 1.0,
  'word.lower()': '09-02-2006',
  'word[-3:]': '006',
  'word[-2:]': '06',
  'word.isupper()': False,
  'word.istitle()': False,
  'word.isdigit()': False,
  'BOS': True,
  '+1:word.lower()': '16:08:49',
  '+1:word.istitle()': False,
  '+1:word.isupper()': False,
  '+1:word.check_dict()': 0,
  '+2:word.lower()': 'gmt',
  '+2:word.istitle()': False,
  '+2:word.isupper()': True},
 {'bias': 1.0,
  'word.lower()': '16:08:49',
  'word[-3:]': ':49',
  'word[-2:]': '49',
  'word.isupper()': False,
  'word.istitle()': False,
  'word.isdigit()': False,
  '-1:word.lower()': '09-02-2006',
  '-1:word.

In [38]:
%%time

crf = CRF(algorithm = "lbfgs",
          c1 = 0.1, 
          c2 = 0.1, 
          max_iterations= 500, 
          all_possible_transitions=True)
try:
    crf.fit(X_train, y_train)
except:
    pass

CPU times: total: 1min 6s
Wall time: 1min 7s


In [39]:
labels = list(crf.classes_)
labels.remove('')
labels

['B_W', 'I_W', 'O']

In [44]:
y_pred = crf.predict(X_test)
metrics.flat_f1_score(y_test, y_pred,
                      average='weighted', labels=labels)
metrics.flat_accuracy_score(y_test, y_pred)

0.9670665079510423

In [45]:
import pickle

pickle.dump(crf, open('crf_model.pkl', 'wb'))

In [46]:
with open('crf_model.pkl', 'rb') as file:
    crf_model = pickle.load(file)
    

In [56]:
def word_segment(text):
    text = text.split(' ')
    text = [word.replace(',' or ':' or '.' or ';', '') for word in text]


In [60]:
text = input()
text = text.replace(',', '')
text = text.replace('.', '')
text = text.replace(';', '')
text = text.replace(':', '')
text = text.replace('?', '')

text = text.split(" ")
text = [(word, -1) for word in text]
untag_text = []
untag_text.append(sent2features(text))
prediction = crf_model.predict(untag_text)
tagged_text = []

for i in range(len(prediction[0])):
    tagged_text.append((text[i][0], prediction[0][i]))

tagged_text

[('Ở', 'B_W'),
 ('đây', 'I_W'),
 ('các', 'B_W'),
 ('bạn', 'B_W'),
 ('chú', 'B_W'),
 ('ý', 'I_W'),
 ('là', 'B_W'),
 ('chúng', 'B_W'),
 ('ta', 'I_W'),
 ('phải', 'B_W'),
 ('padding', 'B_W'),
 ('để', 'B_W'),
 ('đảm', 'B_W'),
 ('bảo', 'I_W'),
 ('các', 'B_W'),
 ('input', 'B_W'),
 ('có', 'B_W'),
 ('cùng', 'B_W'),
 ('độ', 'B_W'),
 ('dài', 'I_W'),
 ('như', 'B_W'),
 ('nhau', 'I_W'),
 ('nhé:', 'B_W')]

In [50]:
token_list = []
pre_word = tagged_text[0][0]

for i in range(1, len(tagged_text)):
    if tagged_text[i][1] == "I_W":
        pre_word += "_" + tagged_text[i][0]
    else:
        token_list.append(pre_word)
        pre_word =  tagged_text[i][0]

token_list.append(pre_word)

token_list

['Ở_đây',
 'các',
 'bạn',
 'chú_ý',
 'là',
 'chúng_ta',
 'phải',
 'padding',
 'để',
 'đảm_bảo',
 'các',
 'input',
 'có',
 'cùng',
 'độ_dài',
 'như_nhau',
 'nhé:']